In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
class DryBeanDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

        # Encode target column
        self.label_encoder = LabelEncoder()
        self.data['Class'] = self.label_encoder.fit_transform(self.data['Class'])

        # Standardize features
        self.scaler = StandardScaler()
        self.data.iloc[:, :-1] = self.scaler.fit_transform(self.data.iloc[:, :-1])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data.iloc[idx, :-1], dtype=torch.float32)
        target = torch.tensor(self.data.iloc[idx, -1], dtype=torch.long)
        return features, target

In [4]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(TwoLayerNet, self).__init__()
        self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(input_dim, hidden_size)
        self.relu = nn.LeakyReLU()
        self.lin2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        return x

In [5]:
criterion = nn.CrossEntropyLoss()

In [16]:
input_dim = 16  # Number of features
hidden_size = 10
num_classes = 7  # Number of unique target classes
model = TwoLayerNet(input_dim, hidden_size, num_classes)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
dataset = DryBeanDataset('Dry_Bean.csv')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)


In [18]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 1.4194
Epoch [2/10], Loss: 1.1964
Epoch [3/10], Loss: 0.9546
Epoch [4/10], Loss: 0.2562
Epoch [5/10], Loss: 0.4527
Epoch [6/10], Loss: 0.5713
Epoch [7/10], Loss: 0.3370
Epoch [8/10], Loss: 0.7900
Epoch [9/10], Loss: 0.7803
Epoch [10/10], Loss: 0.3284


In [19]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in DataLoader(test_data, batch_size=64):
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.9086
